In [40]:
import pandas as pd
from pathlib import Path

metrics_dir = Path("experiments/Metrics")
assert metrics_dir.exists()
metas_dir = Path("experiments/Meta")
assert metas_dir.exists()

In [41]:
import re
from typing import Literal



metric_names = ["LPIPS", "PSNR", "SSIM", "Ref_LPIPS", "Ref_PSNR", "Ref_SSIM", "FID_0", "FID_4", "FID_8"]
experiment_names = ["Finetune_Diffusion", "Neurad_Base_Diffusion", "Neurad_Checkpoint_Training", "Neurad_Training", "Neurad_Finetuned_Diffusion"]
ranks = ["4", "128"]

neurad_experiments = {
    "Neurad_Base_Diffusion", "Neurad_Checkpoint_Training", "Neurad_Training", "Neurad_Finetuned_Diffusion"
}
finetune_experiments = {
    "Finetune_Diffusion"
}


def get_all_metrics_infos(experiment_name):
    infos = []

    if experiment_name == "Finetune_Diffusion":
        for rank in ranks:
            for metric in metric_names:
                path = metrics_dir / f"{experiment_name}-Rank{rank}-{metric}.csv"
                if path.exists():
                    infos.append({"path": path, "rank": rank, "metric": metric})
        
    else:
        for metric in metric_names:
            path = metrics_dir / f"{experiment_name}-{metric}.csv"
            if path.exists():
                infos.append({"path": path, "metric": metric})

    return infos

    

def get_metric_by_name(experiment_name, metric):
    if experiment_name == "Finetune_Diffusion":
        for rank in ranks:
            path = metrics_dir / f"{experiment_name}-Rank{rank}-{metric}.csv"
            if path.exists():
                return pd.read_csv(path)

    path = metrics_dir / f"{experiment_name}-{metric}.csv"    
    if not path.exists():
        return None
    return pd.read_csv(path)


parse_run_pattern1 = re.compile(r"\d+_\d+_(?P<run>\d+).*")
parse_run_pattern2 = re.compile(r"(?P<run>\d+)_\d+_\d+.*")

def filter_column_name(col_name, pattern, value_name = "run"):
    pattern_match = pattern.match(col_name)
    if not pattern_match:
        return col_name

    group_dict = pattern.match(col_name).groupdict()
    return group_dict[value_name]

def parse_run_from_column_name(col_name, pattern = parse_run_pattern1):
    pattern_match = pattern.match(col_name)
    if not pattern_match:
        return col_name

    run = pattern_match.group("run")
    return "run_" + run

def clean_metrics(experiment_name, metrics):
    columns_to_remove = [
        column for column in metrics.columns
        if (column.endswith("__MIN") or column.endswith("__MAX"))
    ] 
    metrics = metrics.drop(columns_to_remove, axis=1)

    if experiment_name in neurad_experiments:
        pattern = parse_run_pattern1    
    else:
        pattern = parse_run_pattern2

    metrics.columns = list(map(lambda c: parse_run_from_column_name(c, pattern), metrics.columns))

    return metrics 

ex_name = "Neurad_Finetuned_Diffusion"
ex_metric = "LPIPS"

metrics = get_metric_by_name(ex_name, ex_metric)
metrics = clean_metrics(ex_name, metrics)
metrics

,Step,run_12146255,run_12146254,run_12146251,run_12146253,run_12146252,run_12146250,run_12146249,run_12146248,run_12146247,run_12146246,run_12146245,run_12146244
0,5000,0.164532,0.229889,0.221506,0.184695,0.156776,0.189177,0.166500,0.222872,0.198615,0.195498,0.229031,0.200974
1,10000,0.156522,0.210406,0.212082,0.181136,0.159866,0.177039,0.364607,0.211682,0.180468,0.353962,0.212767,0.179689
2,15000,0.144070,0.201924,0.200672,0.172626,0.143719,0.168676,0.554299,0.199954,0.167611,NaN,0.200343,0.170008
3,20000,0.165186,0.195418,0.197309,0.166059,0.166120,0.164382,0.572561,0.197184,0.165900,NaN,0.195899,0.166445
4,25000,0.370172,0.193881,0.192417,0.161945,0.191269,0.162209,0.637893,0.190150,0.162846,NaN,0.190852,0.161966
5,30000,0.377656,0.191825,0.483814,0.163782,NaN,0.162221,0.594238,0.192888,0.159024,NaN,0.190852,0.159033
6,35000,0.368122,0.189620,NaN,0.164174,NaN,0.157841,NaN,0.192694,0.162861,NaN,0.188724,0.157851
7,40000,NaN,0.188671,NaN,0.163171,NaN,0.157680,NaN,0.186542,0.404629,NaN,0.188499,0.158182


In [42]:
run_column_pattern = re.compile(r"run_(?P<run>\d+)")

def select_columns_matching_pattern(df, pattern):
    return df.filter(regex=pattern)

In [43]:
improve_direction = {
    "SSIM": "max",
    "PSNR": "max",
    "LPIPS": "min",
    "Ref_SSIM": "max",
    "Ref_PSNR": "max",
    "Ref_LPIPS": "min",
    "FID_0": "min",
    "FID_4": "min",
    "FID_8": "min"
}

def find_best_step(metrics, metric_name):
    runs = select_columns_matching_pattern(metrics, run_column_pattern)
    if improve_direction[metric_name] == "max":
        best_step = runs.idxmax()
    else:
        best_step = runs.idxmin()

    return(best_step)

best_steps = find_best_step(metrics, ex_metric)
best_steps

run_12146255    2
run_12146254    7
run_12146251    4
run_12146253    4
run_12146252    2
run_12146250    7
run_12146249    0
run_12146248    7
run_12146247    5
run_12146246    0
run_12146245    7
run_12146244    6
dtype: int64

In [44]:
def get_best_values(metrics, best_steps):
    subset = select_columns_matching_pattern(metrics, run_column_pattern)
    
    metrics = {}

    for i_col, run in enumerate(subset.columns):
        best_step = best_steps.iloc[i_col]
        best_value = subset.loc[best_step, run]

        try:
            metrics[str(run)] = float(best_value) 
        except BaseException as e:
            print(f"Error: {e}")
            print("Best value", best_value)
            raise e

    return metrics


get_best_values(metrics, best_steps)


{'run_12146255': 0.1440704315900802,
 'run_12146254': 0.1886714845895767,
 'run_12146251': 0.192417487502098,
 'run_12146253': 0.1619447767734527,
 'run_12146252': 0.1437189579010009,
 'run_12146250': 0.157680481672287,
 'run_12146249': 0.1665003001689911,
 'run_12146248': 0.1865417063236236,
 'run_12146247': 0.1590242981910705,
 'run_12146246': 0.1954975128173828,
 'run_12146245': 0.1884991377592086,
 'run_12146244': 0.1578506976366043}

In [58]:
def get_best_metrics(experiment_name, reference_metric: str):
    ref_metrics = get_metric_by_name(experiment_name, reference_metric)
    ref_metrics = clean_metrics(experiment_name, ref_metrics)
    best_steps = find_best_step(ref_metrics, reference_metric)
    

    merged_metrics = {}

    for metrics_info in get_all_metrics_infos(experiment_name):
        
        metrics = pd.read_csv(metrics_info["path"])
        metrics = clean_metrics(experiment_name, metrics)
        values = get_best_values(metrics, best_steps)

        metric_name = metrics_info["metric"]

        for run, value in values.items():
            if run not in merged_metrics:
                merged_metrics[run] = {}

            merged_metrics[run][metric_name] = value

        
    frame = pd.DataFrame.from_dict(merged_metrics, orient="index")
    frame = frame.reset_index(drop=False).rename(columns={"index": "Run"})
    frame["Run"] = frame["Run"].apply(lambda x: int(x.split("_")[1]))

    return frame


ex_experiment = "Finetune_Diffusion"
ex_metric = "LPIPS"
best_metrics = get_best_metrics(ex_experiment, ex_metric)
best_metrics

,Run,LPIPS,PSNR,SSIM,Ref_LPIPS,Ref_PSNR,Ref_SSIM
0,12139546,0.126905,23.917370,0.726831,0.651216,17.515772,0.626276
1,12139541,0.112730,24.471294,0.752596,0.639482,17.801876,0.635347
2,12139540,0.097804,25.030251,0.777955,0.570712,18.130219,0.634849
3,12139536,0.161586,24.860737,0.684128,0.714150,20.247663,0.633088
4,12139535,0.150293,25.359369,0.707100,0.682804,20.393532,0.643994
5,12139532,0.132117,25.839212,0.731585,0.583201,20.574081,0.644031
6,12139528,0.085118,26.400570,0.843583,0.748670,19.491974,0.488186
7,12139525,0.076501,26.971552,0.856741,0.707513,19.122231,0.483250
8,12139524,0.067387,27.579357,0.871178,0.649067,18.767595,0.469647
9,12139520,0.106738,23.471973,0.760450,0.700647,16.885237,0.477032


In [59]:
def get_metas_for_experiment(experiment_name):
    path = metas_dir / f"Meta - {experiment_name}.csv"
    if not path.exists():
        return None

    return pd.read_csv(path)

for experiment_name in experiment_names:
    metas = get_metas_for_experiment(experiment_name)
    if metas is not None:
        print(experiment_name + " - " + str(list(metas.columns)))
    else:
        print(f"No metas for {experiment_name}")

ex_metas = get_metas_for_experiment(ex_experiment)
ex_metas

Finetune_Diffusion - ['Run', 'Scene', 'Rank', 'Noise Strength', 'Controlnet']
Neurad_Base_Diffusion - ['Run', 'Augment Loss Mult', 'Augment Phase Step', 'Noise Start Phase Step', 'Noise Strength']
Neurad_Checkpoint_Training - ['Run', 'Scene']
Neurad_Training - ['Run', 'Model', 'Scene']
Neurad_Finetuned_Diffusion - ['Run', 'Scene']


,Run,Scene,Rank,Noise Strength,Controlnet
0,12139481,53,128,0.3,Yes
1,12139557,53,128,0.2,Yes
2,12139552,53,128,0.1,Yes
3,12139539,28,128,0.3,Yes
4,12139537,28,128,0.1,Yes
5,12139531,16,128,0.3,Yes
6,12139530,16,128,0.2,Yes
7,12139529,16,128,0.1,Yes
8,12139523,11,128,0.3,Yes
9,12139522,11,128,0.2,Yes


In [61]:
def combine_metrics_and_metas(experiment_name, metrics, metas):
    return pd.merge(metrics, metas, on="Run")

ex_combined = combine_metrics_and_metas(ex_experiment, best_metrics, ex_metas)
ex_combined

,Run,LPIPS,PSNR,SSIM,Ref_LPIPS,Ref_PSNR,Ref_SSIM,Scene,Rank,Noise Strength,Controlnet
0,12139546,0.126905,23.917370,0.726831,0.651216,17.515772,0.626276,53,4,0.3,Yes
1,12139541,0.112730,24.471294,0.752596,0.639482,17.801876,0.635347,53,4,0.2,Yes
2,12139540,0.097804,25.030251,0.777955,0.570712,18.130219,0.634849,53,4,0.1,Yes
3,12139536,0.161586,24.860737,0.684128,0.714150,20.247663,0.633088,28,4,0.3,Yes
4,12139535,0.150293,25.359369,0.707100,0.682804,20.393532,0.643994,28,4,0.2,Yes
5,12139532,0.132117,25.839212,0.731585,0.583201,20.574081,0.644031,28,4,0.1,Yes
6,12139528,0.085118,26.400570,0.843583,0.748670,19.491974,0.488186,16,4,0.3,Yes
7,12139525,0.076501,26.971552,0.856741,0.707513,19.122231,0.483250,16,4,0.2,Yes
8,12139524,0.067387,27.579357,0.871178,0.649067,18.767595,0.469647,16,4,0.1,Yes
9,12139520,0.106738,23.471973,0.760450,0.700647,16.885237,0.477032,11,4,0.3,Yes


In [67]:
def load_experiment(experiment_name, ref_metric: str = "LPIPS"):
    metrics = get_best_metrics(experiment_name, ref_metric)
    metas = get_metas_for_experiment(experiment_name)

    return combine_metrics_and_metas(experiment_name, metrics, metas)

experiments = {}
for experiment_name in experiment_names:
    print(f"Loading {experiment_name}")
    loaded_experiment = load_experiment(experiment_name)
    print(f"Success: {isinstance(loaded_experiment, pd.DataFrame)}")

    experiments[experiment_name] = loaded_experiment

Loading Finetune_Diffusion
Success: True
Loading Neurad_Base_Diffusion
Success: True
Loading Neurad_Checkpoint_Training
Success: True
Loading Neurad_Training
Success: True
Loading Neurad_Finetuned_Diffusion
Success: True


In [68]:
experiment_dir = Path("experiments/Experiment")
experiment_dir.mkdir(exist_ok=True)

for experiment_name, experiment in experiments.items():
    experiment.to_csv(experiment_dir / f"{experiment_name}.csv", index=False)

In [ ]:
load_experiment("Neurad_Finetuned_Diffusion")